In [2]:
from compute import Evaluable, anp_math, evaluable_with_unit, Var, get_unit, ureg, coupled_run, buildidpvars, eqvar, unit_conversion_factors
from inputresolver import reassigneq, eqvars, eqsonly, default_out, mdf_order, idx_eqlist
from compute_utils import get_outputs, check_eqs
from utils import invmap
import sympy as sp
import openmdao.api as om
import numpy as np

# trig functions

In [8]:
t = Var('t', 10, unit='', never_output=True) 
g, eq2 = eqvar('g', 1/np.pi*sp.acos(R/a))

# units

In [3]:
from inputresolver import idf_order

In [4]:
g = Var('g', 10, unit='m/s**2', never_output=True) 
m = Var('m', 120, unit='kg') 

In [5]:
sp.lambdify([m,g], m*g, anp_math)(10, 9.81)

98.10000000000001

In [5]:
ev = Evaluable.fromsympy(m*g)

In [6]:
print(ev.fx(100, 9.81)) # here we have to re-enter values for all numbers
print(ev.fx(200, partial='never_output')) # it plugs in the values for variables with never_output = True
print(ev.fx(partial='varval'))

981.0
2000
1200


In [7]:
F, eq1 = eqvar('F', m*g, 'mN')

In [8]:
F = Var('F', unit='mN')
eqs = [(F, m*g)]

## More complex

In [9]:
# From outside
E_service = Var('E_service', 20, unit='W*hr')
# From inside
mu_batt = Var('mu_batt', 30, unit='W*hr/kg')
DOD = Var('DOD', 0.4)
eta_trans = Var('eta_trans', 0.85)
nu_batt = Var('nu_batt', 450, unit='kW*hr/(m**3)')
N = Var('N', 1)
C, eq1 = eqvar('C', E_service/(DOD*N*eta_trans), unit='kW*hr')
m_batt_zero = Var('m_batt_zero', 5, 'kg')
m_batt, eq2 = eqvar('m_batt', C/mu_batt + m_batt_zero, 'kg')
Vol_batt, eq3 = eqvar('Vol_batt', C/nu_batt, 'm**3')

In [10]:
Cq = (20*ureg('W*hr')/(0.4*1*0.85)).to('kW*hr')

In [11]:
Cq/(450*ureg('kW*hr/(m**3)'))

0.000130718954248366 <Unit('meter ** 3')>

In [12]:
eq_list=[eq1, eq2, eq3]
eq_list

[(C, E_service/(DOD*N*eta_trans)),
 (m_batt, C/mu_batt + m_batt_zero),
 (Vol_batt, C/nu_batt)]

In [13]:
eqs = idx_eqlist(eq_list)

In [14]:
ins = getdofs(eqs)

In [15]:
ins

{DOD, E_service, N, eta_trans, m_batt_zero, mu_batt, nu_batt}

In [16]:
eqv = eqvars(eqs)

In [17]:
dout = default_out(eqs)

In [18]:
order = mdf_order(eqv, dout)

In [21]:
order

[0, 2, 1]

In [19]:
dout

{0: C, 1: m_batt, 2: Vol_batt}

In [20]:
idf_order(eqv, dout)

[0, 2, 1]

In [22]:
eqs

{0: (C, E_service/(DOD*N*eta_trans)),
 1: (m_batt, C/mu_batt + m_batt_zero),
 2: (Vol_batt, C/nu_batt)}

In [19]:
prob = om.Problem()
model = prob.model
counter = coupled_run(eqs, order, (), model, model, 0)
buildidpvars(ins, model)
prob.setup()
prob.run_model()

In [20]:
prob.set_val('N', 1)
prob.run_model()

In [22]:
out = get_outputs(eqs, model)
out

{E_service: 20.0,
 DOD: 0.4,
 Vol_batt: 0.000130718954248366,
 m_batt: 6.96078431372549,
 eta_trans: 0.85,
 C: 0.0588235294117647,
 N: 1.0,
 mu_batt: 30.0,
 m_batt_zero: 5.0,
 nu_batt: 450.0}

In [31]:
20.0*E_service.varunit

20.0 <Unit('hour * watt')>

In [51]:
{key: '{:.2f~}'.format((val*key.varunit).to_compact()) for key,val in out.items()}

{E_service: '20.00 W * hr',
 DOD: '0.40',
 Vol_batt: '130718.95 mm ** 3',
 m_batt: '6.96 kg',
 eta_trans: '0.85',
 C: '58.82 W * hr',
 N: '1.00',
 mu_batt: '30.00 hr * mW / g',
 m_batt_zero: '5.00 kg',
 nu_batt: '450.00 hr * kW / m ** 3'}